In [23]:
import pandas as pd
from sklearn.linear_model import Ridge
from scipy.sparse import csr_matrix
import numpy as np
import optuna

In [24]:
SEED = 1
TARGET = 'sales'

BASE = '../data/preprocess/grid_part_1_update.pkl'
DIRECT_RMSE = '../data/submit/Direct_rmse-20200628T030335Z-001/Direct_rmse/20200626_day_by_day_28model_rmse_'
DIRECT_RMSE_ENSEMBLE = '../data/submit/Direct_rmse-20200628T030335Z-001/Direct_rmse/20200626_day_by_day_ensemble_28model_rmse_'
DIRECT_WRMSSE = '../data/submit/Direct_wrmsse-20200628T030424Z-001/Direct_wrmsse/20200626_day_by_day_28model_wrmsse_'
DIRECT_WRMSSE_ENSEMBLE = '../data/submit/Direct_wrmsse-20200628T030424Z-001/Direct_wrmsse/20200626_day_by_day_ensemble_28model_wrmsse_'
DIRREC = '../data/submit/DirRec(多分ここだけF0から始まってるのに注意-20200628T042746Z-001\DirRec(多分ここだけF0から始まってるのに注意/20200628_DirRec_'
BASE_PRED = '../data/submit/Base/20200627_base_model_predict_rmse_'

# WRMSSE計算用pklへのパス
SW_DF    = '../data/master_data/sw_df_update.pkl'
ROLL_MAT = '../data/master_data/roll_mat_df_update.pkl'
EVALUATION   = '../data/raw/sales_train_evaluation.csv'
CALENDAR_CSV = '../data/raw/calendar.csv'
PRICE_CSV    = '../data/raw/sell_prices.csv'
SAMPLE_CSV   = '../data/raw/sample_submission.csv'
TRAIN_RANGE = [1858, 1941]
VALID_RANGE = [1914, 1941]
SPLIT_LIST = [[1858, 1885], [1886, 1913], [1914, 1941]]
EVALUATE_RANGE = [1942, 1969]

In [25]:
grid_df = pd.read_pickle(BASE)
grid_df = grid_df[(grid_df['d'] >= SPLIT_LIST[0][0])]
STORES_IDS = grid_df['store_id']
STORES_IDS = list(STORES_IDS.unique())

grid_df = grid_df[['id', 'd', 'sales']]

In [26]:
def load_stacking_features(grid_df, path, value_name):
    df_tmp = pd.DataFrame()
    print(value_name)
    for val_range in SPLIT_LIST:
        df_preds = pd.read_csv(f'{path}{val_range[0]}.csv')
        df_preds = pd.melt(
            df_preds, 
            id_vars = ["id"], 
            var_name = 'd', 
            value_name = value_name
        )
        df_preds["d"] = df_preds["d"].apply(lambda x : int(x.split("F")[1]) + val_range[0] - 1)
        print(df_preds.head())
        print(df_preds.shape)
        df_tmp = pd.concat([df_tmp, df_preds])
    print("shape: ",df_tmp.shape)
    grid_df = pd.merge(grid_df, df_tmp, on=["id", "d"], how="left")
    
    return grid_df

grid_df = load_stacking_features(grid_df, DIRECT_RMSE, "direct_rmse")
grid_df = load_stacking_features(grid_df, DIRECT_RMSE_ENSEMBLE, "direct_rmse_ensemble")
grid_df = load_stacking_features(grid_df, DIRECT_WRMSSE, "direct_wrmse")
grid_df = load_stacking_features(grid_df, DIRECT_WRMSSE_ENSEMBLE, "direct_wrmse_ensemble")
grid_df = load_stacking_features(grid_df, BASE_PRED, "base_pred")
#grid_df = pd.read_pickle("../data/submit/stacking_concat_data.pkl")

direct_rmse
                              id     d  direct_rmse
0  HOBBIES_1_001_CA_1_evaluation  1858     0.565942
1  HOBBIES_1_002_CA_1_evaluation  1858     0.286879
2  HOBBIES_1_003_CA_1_evaluation  1858     0.366413
3  HOBBIES_1_004_CA_1_evaluation  1858     1.777381
4  HOBBIES_1_005_CA_1_evaluation  1858     0.876924
(853720, 3)
                              id     d  direct_rmse
0  HOBBIES_1_001_CA_1_evaluation  1886     1.002714
1  HOBBIES_1_002_CA_1_evaluation  1886     0.269807
2  HOBBIES_1_003_CA_1_evaluation  1886     0.474564
3  HOBBIES_1_004_CA_1_evaluation  1886     1.843606
4  HOBBIES_1_005_CA_1_evaluation  1886     0.931230
(853720, 3)
                              id     d  direct_rmse
0  HOBBIES_1_001_CA_1_evaluation  1914     0.866332
1  HOBBIES_1_002_CA_1_evaluation  1914     0.329256
2  HOBBIES_1_003_CA_1_evaluation  1914     0.500765
3  HOBBIES_1_004_CA_1_evaluation  1914     1.585723
4  HOBBIES_1_005_CA_1_evaluation  1914     1.019994
(853720, 3)
shape:  (2561160

In [27]:
grid_df.head()

,id,d,sales,direct_rmse,direct_rmse_ensemble,direct_wrmse,direct_wrmse_ensemble,base_pred
0,HOBBIES_1_001_CA_1_evaluation,1858,0.0,0.565942,0.708006,0.636699,0.713258,0.644864
1,HOBBIES_1_002_CA_1_evaluation,1858,0.0,0.286879,0.292619,0.263533,0.278059,0.347659
2,HOBBIES_1_003_CA_1_evaluation,1858,0.0,0.366413,0.444350,0.309666,0.446757,0.430145
3,HOBBIES_1_004_CA_1_evaluation,1858,0.0,1.777381,1.729335,1.696449,1.767311,1.677954
4,HOBBIES_1_005_CA_1_evaluation,1858,1.0,0.876924,0.975472,1.003239,1.004170,0.892836


## WRMSSE計算の準備

In [28]:
sw_df = pd.read_pickle(SW_DF)
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

roll_mat_df = pd.read_pickle(ROLL_MAT)
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)

STORES_IDS = pd.read_csv(EVALUATION)['store_id']
STORES_IDS = list(STORES_IDS.unique())

# ★店舗別のSWを取得
SW_store = {}
for i, store_id in enumerate(STORES_IDS):
    SW_store[store_id] = SW[pd.Index(np.arange(42840))[roll_index.map(lambda x: (x[0] in [2, 7, 8, 11]) and store_id in x[1])]]

# ★店舗別のroll_matを取得
roll_mat_csr_store = {}
for i, store_id in enumerate(STORES_IDS):
    roll_mat_csr_store[store_id] = roll_mat_csr[pd.Index(np.arange(42840))[roll_index.map(lambda x: (x[0] in [2, 7, 8, 11]) and store_id in x[1])], i*3049:(i+1)*3049]
del roll_mat_df

In [29]:
# ★★店舗別にWRMSSEをレベル別で算出する関数を返す関数
def get_wrmsse_per_level_func(store_id, level=None):
    def _wrmsse(preds, y_true, score_only=True, s = S, w = W, sw=SW_store):
        # Function to do quick rollups:
        def _rollup(v, store_id):
            '''
            v - np.array of size (30490 rows, n day columns)
            v_rolledup - array of size (n, 42840)
            '''
            v = v.reshape(28, 3049).T
            return roll_mat_csr_store[store_id]*v #(v.T*roll_mat_csr.T).T

        '''
        preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
        y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
        sequence_length - np.array of size (42840,)
        sales_weight - sales weights based on last 28 days: np.array (42840,)
        '''

        if score_only:
            scores = np.sqrt(
                        np.mean(
                            np.square(_rollup(preds-y_true, store_id))
                                ,axis=1)) * sw[store_id] #<-used to be mistake here
            if level is None:
                return np.sum(scores, axis=0)
            
            score_spans = [
                (0, 1), (1, 4), (4, 11), (11, 3060)          
            ]
            score = np.sum(scores[score_spans[level][0]:score_spans[level][1]], axis=0)
            return score
        else: 
            score_matrix = (np.square(_rollup(preds-y_true)) * np.square(w)[:, None])/ s[:, None]
            score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
            return score, score_matrix

    return _wrmsse

def get_wrmsse_per_level_metric(store_id, level=None):
    wrmsse = get_wrmsse_per_level_func(store_id, level=level)
    def wrmsse_metric(preds, data):

        # this function is calculate for last 28 days to consider the non-zero demand period

        # actual obserbed values / 正解ラベル
        y_true = data.get_label()
        
        if level is None:
            return 'wrmsse-store', wrmsse(preds, y_true), False
        
        return 'wrmsse-level' + str(level), wrmsse(preds, y_true), False
    return wrmsse_metric

In [30]:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T

# Function to calculate WRMSSE:
def wrmsse_metric(preds, y_true, score_only=False, s = S, w = W, sw=SW, level=None):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        scores = np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw
        if level is None:
            return np.sum(scores)/12
        
        score_spans = [
            (0, 1), (1, 4), (4, 14), (14, 17), (17, 24), (24, 33),
            (33, 54), (54, 84), (84, 154), (154, 3203), (3203, 12350), (12350, 42840)           
        ]
        score = np.sum(scores[score_spans[level][0]:score_spans[level][1]], axis=0)/12
        return score
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix

In [31]:
NUM_ITEMS = 30490
DAYS_PRED = 28
def wrmsse(preds, data):
    
    # this function is calculate for last 28 days to consider the non-zero demand period
    
    # actual obserbed values / 正解ラベル
    y_true = data.get_label()
    
    y_true = y_true[-(NUM_ITEMS * DAYS_PRED):]
    preds = preds[-(NUM_ITEMS * DAYS_PRED):]
    # number of columns
    num_col = DAYS_PRED
    
    # reshape data to original array((NUM_ITEMS*num_col,1)->(NUM_ITEMS, num_col) ) / 推論の結果が 1 次元の配列になっているので直す
    reshaped_preds = preds.reshape(num_col, NUM_ITEMS).T
    reshaped_true = y_true.reshape(num_col, NUM_ITEMS).T
    
          
    # train = weight_mat_csr*np.c_[reshaped_preds, reshaped_true]
    train = roll_mat_csr*np.c_[reshaped_preds, reshaped_true]
    """
    score = np.sum(
                np.sqrt(
                    np.mean( # ★ RMSSEの分子 1/h * SUM_t=n+1~n+h (Y_t - Y^_t)^2 に対応
                        np.square(
                            train[:,:num_col] - train[:,num_col:]) # ★ 予測誤差（Y_t - Y^_t）
                        ,axis=1)
                    
                    / weight1) * weight2)
    """
    score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(train[:,:num_col] - train[:,num_col:]))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    return 'wrmsse', score, False

## optuna

In [32]:
def objective(trial):

    direct_rmse = trial.suggest_uniform('direct_rmse', 0, 1)
    direct_rmse_ensemble = trial.suggest_uniform('direct_rmse_ensemble', 0, 1)
    direct_wrmse = trial.suggest_uniform('direct_wrmse', 0, 1)
    direct_wrmse_ensemble = trial.suggest_uniform('direct_wrmse_ensemble', 0, 1)
    base_pred = trial.suggest_uniform('base_pred', 0, 1)
    
    weight_sum = (
        direct_rmse + 
        direct_rmse_ensemble + direct_wrmse + 
        direct_wrmse_ensemble + 
        base_pred
    )
    wrmsse_list = []
    for val_range in SPLIT_LIST:
        valid_mask = ((grid_df['d'] >= val_range[0]) & (grid_df['d'] <= val_range[1]))
        valid_data = grid_df[valid_mask].drop(["id", "d", TARGET], axis=1)
        pred = (
            direct_rmse*valid_data["direct_rmse"] + 
            direct_rmse_ensemble*valid_data["direct_rmse_ensemble"] + 
            direct_wrmse*valid_data["direct_wrmse"] + 
            direct_wrmse_ensemble*valid_data["direct_wrmse_ensemble"] +
            base_pred*valid_data["base_pred"]
        ) / weight_sum
        
        all_preds = pd.DataFrame()
        valid_result_df = grid_df[valid_mask][["id", "d"]]
        valid_result_df["pred"] = pred
        for PREDICT_DAY in range(1, 28+1):
            day_to_predict = val_range[0]-1+PREDICT_DAY
            tmp = valid_result_df[valid_result_df["d"]==day_to_predict].rename(columns={"pred":"pred_" + str(day_to_predict)})
            tmp = tmp[["id", "pred_" + str(day_to_predict)]]

            if 'id' in list(all_preds):
                all_preds = all_preds.merge(tmp, on=['id'], how='left')
            else:
                all_preds = tmp.copy()
        # Predict
        sales =  pd.read_csv(EVALUATION)
        base_id_list = sales["id"].to_list()
        all_preds_val = pd.DataFrame({"id":base_id_list})
        all_preds_val = pd.merge(all_preds_val, all_preds, on="id", how="left")
        all_preds_val = all_preds_val.fillna(0)
        # Ground truth:
        dayCols = ["d_{}".format(i) for i in range(val_range[0], val_range[1]+1)]
        y_true = sales[dayCols]
        valid_preds = all_preds_val.drop("id", axis=1)
        score = wrmsse_metric(valid_preds, y_true, score_only=True)
        wrmsse_list.append(score)
        #print("validation : "+str(val_range[0])+" - "+str(val_range[1]) + " : " +str(score))
    print(wrmsse_list)
    return np.mean(wrmsse_list)

In [104]:
study = optuna.create_study()
study.optimize(objective, n_trials=500)

[0.5961367997410177, 0.48874944110924096, 0.5463977919915414]


[I 2020-06-29 23:53:24,612] Finished trial#0 resulted in value: 0.5437613442806. Current best value is 0.5437613442806 with parameters: {'direct_rmse': 0.9129695275009985, 'direct_rmse_ensemble': 0.1162453505756772, 'direct_wrmse': 0.15628675144769366, 'direct_wrmse_ensemble': 0.7811088653738347, 'base_pred': 0.8663688903590022}.


[0.593182527945862, 0.4933046772209953, 0.5435163700304206]


[I 2020-06-29 23:53:36,977] Finished trial#1 resulted in value: 0.5433345250657593. Current best value is 0.5433345250657593 with parameters: {'direct_rmse': 0.8756688380294786, 'direct_rmse_ensemble': 0.4900179813078146, 'direct_wrmse': 0.9764127601260632, 'direct_wrmse_ensemble': 0.5696939397549384, 'base_pred': 0.4482439949096214}.


[0.5966340105845656, 0.48539490514754924, 0.5488751421549453]


...

[I 2020-06-30 01:34:44,370] Finished trial#497 resulted in value: 0.5428017852471149. Current best value is 0.5426543617180987 with parameters: {'direct_rmse': 0.43620693105558084, 'direct_rmse_ensemble': 0.00035695939630667195, 'direct_wrmse': 0.42979567152384984, 'direct_wrmse_ensemble': 0.8845383929960402, 'base_pred': 0.366715757652114}.


[0.5945378150190164, 0.488253816265962, 0.5461779447792969]


[I 2020-06-30 01:34:57,326] Finished trial#498 resulted in value: 0.5429898586880918. Current best value is 0.5426543617180987 with parameters: {'direct_rmse': 0.43620693105558084, 'direct_rmse_ensemble': 0.00035695939630667195, 'direct_wrmse': 0.42979567152384984, 'direct_wrmse_ensemble': 0.8845383929960402, 'base_pred': 0.366715757652114}.


[0.5947245205062281, 0.4901949470782818, 0.5452098755725205]


[I 2020-06-30 01:35:10,210] Finished trial#499 resulted in value: 0.5433764477190102. Current best value is 0.5426543617180987 with parameters: {'direct_rmse': 0.43620693105558084, 'direct_rmse_ensemble': 0.00035695939630667195, 'direct_wrmse': 0.42979567152384984, 'direct_wrmse_ensemble': 0.8845383929960402, 'base_pred': 0.366715757652114}.


In [12]:
sub = pd.read_csv(SAMPLE_CSV)

In [13]:
sub

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
weight_dic = {'direct_rmse': 0.43620693105558084, 'direct_rmse_ensemble': 0.00035695939630667195, 'direct_wrmse': 0.42979567152384984, 'direct_wrmse_ensemble': 0.8845383929960402, 'base_pred': 0.366715757652114}
sum_weight = np.sum(list(weight_dic.values()))
for key in weight_dic.keys():
    weight_dic[key] = weight_dic[key]/sum_weight

In [18]:
weight_dic

{'direct_rmse': 0.20598985001617023,
 'direct_rmse_ensemble': 0.00016856681375772306,
 'direct_wrmse': 0.20296226311799753,
 'direct_wrmse_ensemble': 0.4177052631095909,
 'base_pred': 0.17317405694248367}

## create submit

In [34]:
DIRECT_RMSE = "../data/submit/Direct_rmse-20200628T030335Z-001/Direct_rmse/submit/20200625_day_by_day_28model_rmse_submit.csv"
DIRECT_RMSE_ENSEMBLE = "../data/submit/Direct_rmse-20200628T030335Z-001/Direct_rmse/submit/20200625_day_by_day_ensemble_28model_rmse_submit.csv"
DIRECT_WRMSE = "../data/submit/Direct_wrmsse-20200628T030424Z-001/Direct_wrmsse/submit/20200628_day_by_day_28model_wrmsse_submit.csv"
DIRECT_WRMSE_ENSEMBLE = "../data/submit/Direct_wrmsse-20200628T030424Z-001/Direct_wrmsse/submit/20200628_day_by_day_ensemble_28model_wrmsse_submit.csv"
BASE_PRED = "../data/submit/Base/20200627_base_model_predict_rmse_submit.csv"

SW_DF    = '../data/master_data/sw_df_update.pkl'
ROLL_MAT = '../data/master_data/roll_mat_df_update.pkl'
EVALUATION   = '../data/raw/sales_train_evaluation.csv'
CALENDAR_CSV = '../data/raw/calendar.csv'
PRICE_CSV    = '../data/raw/sell_prices.csv'
SAMPLE_CSV   = '../data/raw/sample_submission.csv'

In [35]:
path_list = [DIRECT_RMSE, DIRECT_RMSE_ENSEMBLE, DIRECT_WRMSE, DIRECT_WRMSE_ENSEMBLE, BASE_PRED]

In [37]:
grid_df = pd.read_csv(SAMPLE_CSV)
F_columns = grid_df.columns[1:]
for key, path in zip(weight_dic.keys(), path_list):
    print(f'{key}, {path}')
    print(pd.read_csv(path).head())
    grid_df.loc[:, F_columns] = grid_df.loc[:, F_columns] + pd.read_csv(path)[F_columns] * weight_dic[key]

direct_rmse, ../data/submit/Direct_rmse-20200628T030335Z-001/Direct_rmse/submit/20200625_day_by_day_28model_rmse_submit.csv
                              id        F1        F2        F3        F4  \
0  HOBBIES_1_001_CA_1_validation  0.793877  0.747731  0.783910  0.817366   
1  HOBBIES_1_002_CA_1_validation  0.293176  0.275680  0.249223  0.240530   
2  HOBBIES_1_003_CA_1_validation  0.475781  0.412732  0.414113  0.432541   
3  HOBBIES_1_004_CA_1_validation  1.631193  1.340382  1.376494  1.368842   
4  HOBBIES_1_005_CA_1_validation  1.064908  0.916209  0.988002  1.026198   

         F5        F6        F7        F8        F9  ...       F19       F20  \
0  0.975276  1.282243  1.164445  0.977888  0.884113  ...  0.915187  1.088486   
1  0.261211  0.305128  0.323074  0.208948  0.232060  ...  0.286546  0.370702   
2  0.572805  0.774625  0.747049  0.437051  0.470890  ...  0.561403  0.724256   
3  1.907489  2.636012  3.299511  1.823796  1.474890  ...  1.658987  2.681150   
4  1.227318  1.4963

In [38]:
grid_df.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.807756,0.741655,0.748132,0.776561,0.909310,1.112579,1.210648,0.938844,0.890970,...,0.911621,1.081210,1.024401,0.861332,0.768087,0.756233,0.810257,0.856802,1.017744,1.007515
1,HOBBIES_1_002_CA_1_validation,0.272845,0.251947,0.239538,0.231086,0.279042,0.327305,0.342047,0.232609,0.233508,...,0.262698,0.336393,0.341253,0.237781,0.211615,0.237992,0.232915,0.276310,0.330253,0.332338
2,HOBBIES_1_003_CA_1_validation,0.457763,0.393904,0.409750,0.415691,0.520235,0.694397,0.696273,0.469646,0.434739,...,0.572739,0.708206,0.746077,0.530459,0.472738,0.471479,0.467387,0.541501,0.721121,0.758910
3,HOBBIES_1_004_CA_1_validation,1.647189,1.352557,1.341442,1.405752,1.800945,2.556892,3.118023,1.851418,1.569055,...,1.683710,2.569379,3.124821,1.868699,1.463282,1.432093,1.370246,1.819624,2.997349,3.236056
4,HOBBIES_1_005_CA_1_validation,1.104109,0.937870,0.962918,0.995457,1.186055,1.460608,1.561167,1.253515,1.164771,...,1.168757,1.463641,1.546373,1.159430,1.009049,1.016034,1.023412,1.182425,1.371864,1.445809


In [39]:
grid_df.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.483718,0.451253,0.403633,0.395878,0.467109,0.521447,0.499351,0.448230,0.431303,...,0.559999,0.751798,0.820120,0.594436,0.636918,0.640537,0.532880,0.571344,0.676817,0.678207
60976,FOODS_3_824_WI_3_evaluation,0.266085,0.263589,0.264205,0.263389,0.298943,0.329521,0.302944,0.258298,0.274054,...,0.304207,0.363225,0.355742,0.278842,0.287542,0.291085,0.278467,0.278774,0.354066,0.325234
60977,FOODS_3_825_WI_3_evaluation,0.655374,0.575464,0.550755,0.553673,0.634670,0.767969,0.720578,0.685792,0.565882,...,0.781769,0.999173,1.119172,0.817423,0.857576,0.832742,0.694843,0.738320,0.824017,0.872887
60978,FOODS_3_826_WI_3_evaluation,1.016921,1.016179,0.928760,0.958401,1.141144,1.223409,1.113270,1.051579,0.954005,...,1.130687,1.334400,1.308524,1.056466,1.122652,1.040590,0.988834,1.157301,1.283711,1.190524
60979,FOODS_3_827_WI_3_evaluation,1.298568,1.260714,1.193358,1.226066,1.511079,1.758615,1.432493,1.384550,1.277536,...,1.443503,1.784770,1.671093,1.312866,1.410163,1.378999,1.276329,1.574857,1.897890,1.706503


In [ ]:
grid_df.to_csv("../data/submit/ensemble_5model_weight_by_optuna.csv" ,index=False)